In [1]:
import pandas as pd, numpy as np, os, glob
pd.options.mode.chained_assignment = None

In [2]:
Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_2693_PeriHD.csv')
IntraHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_1124_IntraHD.csv')
Test = pd.read_csv('/Users/monoaki/vs_code/亞東/encrypt/baseline_01_sample_500-test_periHD.csv')
IntraHD_Test = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/IntraHD_ExternalData_05-17.csv')

/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_33750/4058901753.py:1: DtypeWarning: Columns (6,7,8,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_2693_PeriHD.csv')
/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_33750/4058901753.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  IntraHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_1124_IntraHD.csv')


In [3]:
full_DataFrame = pd.read_csv('../../../encrypt/periHD2016_20_en.csv')

/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_33750/3472545216.py:1: DtypeWarning: Columns (3,5,6,7,8,9,10,11,15,16,17,18,25,26,27,28,29,30,31,41,45,46,47,48,49,50,52,53,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  full_DataFrame = pd.read_csv('../../../encrypt/periHD2016_20_en.csv')


In [4]:
IntraHD_Train['處置其他+症狀處置'] = IntraHD_Train['處置其他']+'||'+IntraHD_Train['症狀處置']
print(IntraHD_Train.shape)
IntraHD_Train = IntraHD_Train.drop_duplicates(subset=['處置其他+症狀處置'])
print(IntraHD_Train.shape)
print(Train.shape)
Train = Train.drop_duplicates(subset=['處置其他結束'])
# Train = pd.concat([Train,Train.sample(307, random_state=123)])
Train['透析時間長度'] = pd.to_datetime(Train['透析時間結束'])-pd.to_datetime(Train['透析開始時間'])
Test['透析時間長度'] = pd.to_datetime(Test['透析時間結束'])-pd.to_datetime(Test['透析開始時間'])
print(Train.shape)
print(Test.shape)

(1452843, 25)
(1124, 25)
(61160, 64)
(2386, 65)
(500, 66)


In [5]:
Train_ID = Train['ID'].astype(str).drop_duplicates()
Test_ID = Test['ID'].astype(str).drop_duplicates()
overall_ID = pd.concat([Train_ID,Test_ID]).drop_duplicates()
print(Train_ID.shape, Test_ID.shape, overall_ID.shape)
full_DataFrame_Train = pd.merge(Train_ID, full_DataFrame, on='ID', how='inner')
full_DataFrame_Test = pd.merge(Test_ID, full_DataFrame, on='ID', how='inner')
full_DataFrame_Overall = pd.merge(overall_ID, full_DataFrame, on='ID', how='inner')
full_DataFrame_Overall['ID'] = full_DataFrame_Overall['ID'].astype(str)

(1259,) (357,) (1382,)


In [6]:

First_dcit = pd.DataFrame([], columns=full_DataFrame_Overall.columns.to_list())
for idx, ID_ in enumerate(overall_ID.to_list()):
    First_Filter_ID_ = full_DataFrame_Overall[full_DataFrame_Overall['ID']==ID_]
    First_Filter_ID_['洗腎紀錄時間'] = pd.to_datetime(First_Filter_ID_['洗腎紀錄時間']).copy(deep=True)
    First_Filter_DateTime = First_Filter_ID_.sort_values(by=['洗腎紀錄時間'])
    Get_First_Records = First_Filter_DateTime.iloc[0].to_frame().transpose()
    First_dcit = pd.concat([First_dcit, Get_First_Records])

In [10]:
# 基本資料 Variables
age_df = pd.read_csv('../../../encrypt/Base2016_20_en.csv')
age_df_overall = pd.merge(age_df, First_dcit, on='ID', how='inner')
age_df_overall['生日'] = pd.to_datetime(age_df_overall['生日']).copy(deep=True)
age_df_overall['age'] = ((age_df_overall['洗腎紀錄時間']-age_df_overall['生日'])/365).astype(str).str.split(' days').str[0]
age_df_overall['透析時間長度'] = pd.to_datetime(age_df_overall['透析時間結束'])-pd.to_datetime(age_df_overall['透析開始時間'])
# Laboratory results
lab_df = pd.concat([pd.read_csv('../../../encrypt/backup/Lab2016_18_en.csv'),pd.read_csv('../../../encrypt/backup/Lab2019_20_en.csv')])
lab_df['報告日'] = pd.to_datetime(lab_df['報告日']).dt.to_period('D')

In [ ]:
First_dict_AV = First_dcit[['ID','FISTULA','GRAFT','DLC']]

In [12]:
lab_dict = {'WBC count, 10^3/μL':[],'Hemoglobin, g/ dL':[],'Hematocrit, %':[],'MCV, fL':[],'MCH, pg':[],'MCHC, g/dL':[],
            'PLT count, 10^3/ μL':[],'Calcium, mg/dL':[],'Phosphate, mg/ dL':[],'Glucose, mg/dL':[],'BUN, mg/dL':[],'Creatinine, mg/ dL':[],
            'Albumin, g/dL':[], 'Electrolyte Sodium, mmol/L':[],'Potassium, mmol/L':[], 'Serum iron, μg/ dL':[], 'Ferritin, ng/mL':[],
            'Parathyroid hormone, pg/ mL':[],'Urea Reduction Rate, %':[],'ALT (SGPT)':[],'IRON/TIBC':[],'HBV, n (%)':[],'HCV, n (%)':[],}
lab_df_get_first_datetime = lab_df.sort_values(by=['ID','報告日'])
lab_df_get_first_datetime['ID'] = lab_df_get_first_datetime['ID'].astype(str)
print(lab_df_get_first_datetime.shape)
lab_df_get_first_datetime = pd.merge(First_dcit['ID'], lab_df_get_first_datetime, on='ID', how='inner')
print(lab_df_get_first_datetime.shape)

(1272247, 4)
(925848, 4)


In [14]:
# merge 篩選區
age_df_overall = age_df_overall
age_df_Internal = pd.merge(Train_ID, age_df_overall, on='ID', how='inner')
age_df_External =pd.merge(Test_ID, age_df_overall, on='ID', how='inner')
print(age_df_overall.shape, age_df_Internal.shape, age_df_External.shape)
lab_df_overall = lab_df_get_first_datetime
lab_df_Internal = pd.merge(Train_ID, lab_df_get_first_datetime, on='ID', how='inner')
lab_df_External =pd.merge(Test_ID, lab_df_get_first_datetime, on='ID', how='inner')
print(lab_df_overall.shape, lab_df_Internal.shape, lab_df_External.shape)

(1382, 68) (1259, 68) (357, 68)
(925848, 4) (795253, 4) (469708, 4)


In [92]:
# lab_dict = {'WBC count, 10^3/μL':[],'Hemoglobin, g/ dL':[],'Hematocrit, %':[],'MCV, fL':[],'MCH, pg':[],'MCHC, g/dL':[],
#             'PLT count, 10^3/ μL':[],'Calcium, mg/dL':[],'Phosphate, mg/ dL':[],'Glucose, mg/dL':[],'BUN, mg/dL':[],'Creatinine, mg/ dL':[],
#             'Albumin, g/dL':[], 'Electrolyte Sodium, mmol/L':[],'Potassium, mmol/L':[], 'Serum iron, μg/ dL':[], 'Ferritin, ng/mL':[],
#             'Parathyroid hormone, pg/ mL':[],'Urea Reduction Rate, %':[],'ALT (SGPT)':[],'IRON/TIBC':[],'HBV, n (%)':[],'HCV, n (%)':[],}
lab_dict = {'WBC count, 10^3/μL':[],'MCV, fL':[],'MCH, pg':[],'MCHC, g/dL':[],
            'PLT count, 10^3/ μL':[],'Calcium, mg/dL':[],'Phosphate, mg/ dL':[],'Glucose, mg/dL':[],'BUN, mg/dL':[],'Creatinine, mg/ dL':[],
            'Albumin, g/dL':[], 'Serum iron, μg/ dL':[], 'Ferritin, ng/mL':[],
            'Parathyroid hormone, pg/ mL':[],'ALT (SGPT)':[],'IRON/TIBC':[]}
dataframe = lab_df_External
dataframe = dataframe.sort_values(by=['ID','報告日'])
def get_lab_value(dataframe, name='WBC'):
    if name in ['Glucose']:
        value_list = dataframe[dataframe['檢驗名稱'].str.contains(name)].drop_duplicates(subset='ID')
    else:
        value_list = dataframe[dataframe['檢驗名稱']==name].drop_duplicates(subset='ID')
    value_list = value_list.dropna()
    value_list = value_list.apply(pd.to_numeric, errors='coerce')
    return "{} ({})".format(round(np.mean(value_list['DATANUM']),1),round(np.std(value_list['DATANUM']),1))
lab_list = ['WBC','MCV','MCH','MCHC','Platelet','Ca','P  (Phosphorus)','Glucose','BUN','Creatinine','ALBUMIN',
            'Iron','Ferritin','Intact PTH','ALT (SGPT)','IRON/TIBC']
for idx,lab_type in enumerate(lab_list):
    lab_dict_col = list(lab_dict.keys())
    lab_dict[lab_dict_col[idx]].append(get_lab_value(lab_df_Internal, name = lab_type))
    lab_dict[lab_dict_col[idx]].append(get_lab_value(lab_df_External, name = lab_type))
    lab_dict[lab_dict_col[idx]].append(get_lab_value(lab_df_overall, name = lab_type))

In [57]:
# ["Hemoglobin","Hematocrit","Electrolyte_Sodium","Potassium","Urea_Reduction_Rate",'HBV','HCV']

,ID,報告日,檢驗名稱,DATANUM
36498,00189F515,2017-01-26,WBC,19.41
159086,00190A717,2016-01-21,WBC,9.92
425274,00190S010,2016-01-21,WBC,6.18
46604,00258U121,2018-06-14,WBC,5.1
253844,0025GF525,2016-01-21,WBC,4.79
...,...,...,...,...
140083,97898070,2018-04-20,WBC,2.82
195373,97912474,2018-08-30,WBC,3.9
153000,98733181,2017-12-11,WBC,9.2
349459,98883484,2018-08-15,WBC,8.61


In [102]:
def per_hours(dataframe):
    time_delta = [4 if time.components.hours>5  else time.components.hours for time in pd.to_timedelta(dataframe.dropna())]
    mean = round(np.mean(time_delta),1)
    return f"{mean} ({round(np.std(time_delta),1)})"

def weight_calc(dataframe):
    dataframe_pos = dataframe[['體重1開始','體重機器顯示','體重結束']]
    dataframe_pos = dataframe_pos.apply(pd.to_numeric, errors='coerce')
    dataframe_pos = dataframe_pos.dropna()
    dataframe_pos = dataframe_pos.reset_index()
    dataframe_pos['洗後體重'] = dataframe_pos['體重1開始']-dataframe_pos['體重機器顯示']
    for i in range(len(dataframe_pos)):
        if dataframe_pos.loc[i,'洗後體重'] <30:
            dataframe_pos.loc[i,'洗後體重'] = dataframe_pos.loc[i,'體重結束']
        if dataframe_pos.loc[i,'洗後體重'] >100:
            dataframe_pos.loc[i,'洗後體重'] = dataframe_pos.loc[i,'體重結束']
    dataframe_pos['乾體重'] = dataframe_pos['體重結束']
    return dataframe_pos

def VA(dataframe):
    FISTULA = "{} ({}%)".format(len(dataframe['FISTULA'].dropna()),round(len(dataframe['FISTULA'].dropna())/len(dataframe)*100,1))
    GRAFT = "{} ({}%)".format(len(dataframe['GRAFT'].dropna()),round(len(dataframe['GRAFT'].dropna())/len(dataframe)*100,1))
    Catheter = dataframe[(dataframe['DLC'].notna())|(dataframe['PERMCATH'].notna())][['DLC','PERMCATH']]
    Catheter = "{} ({}%)".format(len(Catheter),round(len(Catheter)/len(dataframe)*100,1))
    return [FISTULA, GRAFT, Catheter]

def get_BP(dataframe):
    dataframe = dataframe[['NEW開始血壓SBP','NEW開始血壓DBP','開始脈搏','結束血壓SBP','結束血壓DBP','結束脈搏']]
    dataframe = dataframe.apply(pd.to_numeric, errors='coerce')
    dataframe = dataframe.dropna().astype(float)
    dataframe = dataframe[(dataframe['NEW開始血壓SBP']>=40) & (dataframe['NEW開始血壓SBP']<=240) & (dataframe['結束血壓SBP']>=40) & (dataframe['結束血壓SBP']<=240)]
    dataframe = dataframe[(dataframe['NEW開始血壓DBP']>=30) & (dataframe['NEW開始血壓DBP']<=140) & (dataframe['結束血壓DBP']>=30) & (dataframe['結束血壓DBP']<=140)]
    return dataframe

Table_1_Variables = {"Variables":['Participants, n', 'Age, years', 'Men, n(%)', 'Weight, kg'],
                     "Internal Data":[1259, 
                                      f"{round(np.mean(age_df_Internal['age'].astype(int).values),1)} ({round(age_df_Internal['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_Internal['性別']=='男')} ({round(sum(age_df_Internal['性別']=='男')/len(age_df_Internal)*100,1)}%)",
                                      f"{round(np.mean(age_df_Internal['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_Internal['體重1開始'].astype(float).std(),1)})"
                                      ],
                     "External Data":[357,
                                      f"{round(np.mean(age_df_External['age'].astype(int).values),1)} ({round(age_df_External['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_External['性別']=='男')} ({round(sum(age_df_External['性別']=='男')/len(age_df_External)*100,1)}%)",
                                      f"{round(np.mean(age_df_External['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_External['體重1開始'].astype(float).std(),1)})"
                                      ],
                     "Overall Data":[1382,
                                      f"{round(np.mean(age_df_overall['age'].astype(int).values),1)} ({round(age_df_overall['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_overall['性別']=='男')} ({round(sum(age_df_overall['性別']=='男')/len(age_df_overall)*100,1)}%)",
                                      f"{round(np.mean(Test['體重1開始'].fillna(0).astype(float)),1)} ({round(Test['體重1開始'].astype(float).std(),1)})"
                                      ]}

# 暫時移除Blood flow rate：來源於IntraHD
Table_1_Dialysis_information = {"Variables":['Hours per session, hour','Dry body wieght, kg','Pre-dialysis weight, kg','post-dialysis, kg'],
                                "Internal Data":[per_hours(Train['透析時間長度']),
                                                 f"{round(np.mean(weight_calc(age_df_Internal)['乾體重']),1)} ({round(np.std(weight_calc(age_df_Internal)['乾體重']),1)})",
                                                 f"{round(np.mean(age_df_Internal['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_Internal['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(age_df_Internal)['洗後體重']),1)} ({round(np.std(weight_calc(age_df_Internal)['洗後體重']),1)})",
                                                 ],
                                "External Data":[per_hours(age_df_External['透析時間長度']),
                                                 f"{round(np.mean(weight_calc(age_df_External)['乾體重']),1)} ({round(np.std(weight_calc(age_df_External)['乾體重']),1)})",
                                                 f"{round(np.mean(age_df_External['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_External['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(age_df_External)['洗後體重']),1)} ({round(np.std(weight_calc(age_df_External)['洗後體重']),1)})",
                                                 ],
                                "Overall Data":[per_hours(age_df_overall['透析時間長度']),
                                                 f"{round(np.mean(weight_calc(age_df_overall)['乾體重']),1)} ({round(np.std(weight_calc(age_df_overall)['乾體重']),1)})",
                                                 f"{round(np.mean(age_df_overall['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_overall['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(age_df_overall)['洗後體重']),1)} ({round(np.std(weight_calc(age_df_overall)['洗後體重']),1)})",
                                                 ]} 

Table_1_Vascular_access = {"Variables": ['AV-Fistula, n(%)','AV-Graft, n(%)','Catheter, n(%)'],
                           "Internal Data":VA(age_df_overall),
                           "External Data":VA(age_df_Internal),
                           "Overall Data":VA(age_df_External)}

Table_1_Blood_pressures = {"Variables":['Start-SBP, mmHg','Start-DBP, mmHg','Start-HR, beat/min','End-SBP, mmHg','End-DBP, mmHg','End-HR, beat/min'],
                           "Internal Data":[f"{round(np.mean(get_BP(age_df_Internal)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(age_df_Internal)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(age_df_Internal)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['開始脈搏']),1)} ({round(np.std(get_BP(age_df_Internal)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['結束血壓SBP']),1)} ({round(np.std(get_BP(age_df_Internal)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['結束血壓DBP']),1)} ({round(np.std(get_BP(age_df_Internal)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['結束脈搏']),1)} ({round(np.std(get_BP(age_df_Internal)['結束脈搏']),1)})",
                                            ],
                           "External Data":[f"{round(np.mean(get_BP(age_df_External)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(age_df_External)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(age_df_External)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['開始脈搏']),1)} ({round(np.std(get_BP(age_df_External)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['結束血壓SBP']),1)} ({round(np.std(get_BP(age_df_External)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['結束血壓DBP']),1)} ({round(np.std(get_BP(age_df_External)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['結束脈搏']),1)} ({round(np.std(get_BP(age_df_External)['結束脈搏']),1)})",
                                            ],
                           "Overall Data":[f"{round(np.mean(get_BP(age_df_overall)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(age_df_overall)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(age_df_overall)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['開始脈搏']),1)} ({round(np.std(get_BP(age_df_overall)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['結束血壓SBP']),1)} ({round(np.std(get_BP(age_df_overall)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['結束血壓DBP']),1)} ({round(np.std(get_BP(age_df_overall)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['結束脈搏']),1)} ({round(np.std(get_BP(age_df_overall)['結束脈搏']),1)})",
                                            ]}

In [103]:
pd.concat([pd.DataFrame(Table_1_Variables),pd.DataFrame(Table_1_Vascular_access),pd.DataFrame(Table_1_Dialysis_information), pd.DataFrame(Table_1_Blood_pressures),pd.DataFrame(lab_dict).transpose().reset_index().rename(columns={'index':'Variables',0:'Internal Data',1:'External Data', 2:'Overall Data'})], axis=0)

,Variables,Internal Data,External Data,Overall Data
0,"Participants, n",1259,357,1382
1,"Age, years",64.6 (13.1),61.0 (11.8),64.3 (13.0)
2,"Men, n(%)",705 (56.0%),208 (58.3%),786 (56.9%)
3,"Weight, kg",59.6 (19.1),60.5 (18.2),63.0 (14.2)
0,"AV-Fistula, n(%)",663 (48.0%),595 (47.3%),213 (59.7%)
1,"AV-Graft, n(%)",133 (9.6%),124 (9.8%),26 (7.3%)
2,"Catheter, n(%)",522 (37.8%),482 (38.3%),123 (34.5%)
0,"Hours per session, hour",3.4 (0.7),2.8 (1.1),2.8 (1.1)
1,"Dry body wieght, kg",60.2 (16.5),62.7 (14.6),61.4 (16.2)
2,"Pre-dialysis weight, kg",59.6 (19.1),60.5 (18.2),59.8 (19.1)
